<a href="https://colab.research.google.com/github/ThuanPhong0126/CS114.K21/blob/master/Sarcasm_Headlines_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files 
files.upload()



Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"thuanphong","key":"ffc8cd53940c5c0bf1c827618250572d"}'}

In [0]:
!pip install kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets list -s headlines

ref                                                          title                                             size  lastUpdated          downloadCount  
-----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
therohk/million-headlines                                    A Million News Headlines                          20MB  2020-01-15 15:30:53          18161  
rmisra/news-headlines-dataset-for-sarcasm-detection          News Headlines Dataset For Sarcasm Detection       3MB  2019-07-03 23:52:57          12232  
therohk/india-headlines-news-dataset                         News Headlines Of India                           68MB  2019-04-12 02:46:04           3630  
aaron7sun/stocknews                                          Daily News for Stock Market Prediction             6MB  2019-11-13 18:22:52          29469  
rmisra/news-category-dataset                                 News Category D

In [0]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [0]:
!unzip /content/news-headlines-dataset-for-sarcasm-detection.zip -d /content/

Archive:  /content/news-headlines-dataset-for-sarcasm-detection.zip
  inflating: /content/Sarcasm_Headlines_Dataset.json  
  inflating: /content/Sarcasm_Headlines_Dataset_v2.json  


In [0]:
import pandas as pd

In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [0]:
f1=pd.read_json('/content/Sarcasm_Headlines_Dataset.json',lines=True)

In [0]:
f2=pd.read_json('/content/Sarcasm_Headlines_Dataset_v2.json',lines=True)

In [0]:
df=pd.concat([f1,f2],axis=0,sort=False)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [0]:
df=df.drop('article_link',axis=1)
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [0]:
X=df['headline']
y=df['is_sarcastic']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [0]:
X=cv.fit_transform(X)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
nb=MultinomialNB()

In [0]:
nb.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
pre=nb.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report

In [0]:
print(confusion_matrix(y_test,pre))

[[8189  784]
 [ 948 6678]]


In [0]:
print(classification_report(y_test,pre))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      8973
           1       0.89      0.88      0.89      7626

    accuracy                           0.90     16599
   macro avg       0.90      0.89      0.89     16599
weighted avg       0.90      0.90      0.90     16599



In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
from sklearn.pipeline import Pipeline

In [0]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),  
    ('classifier', MultinomialNB()),  
])

In [0]:
X_train,X_test,y_train,y_test=train_test_split(df['headline'],df['is_sarcastic'],test_size=0.3,random_state=101)

In [0]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
predictions=pipeline.predict(X_test)

In [0]:
print(confusion_matrix(y_test,predictions))

[[8415  558]
 [1369 6257]]


In [0]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      8973
           1       0.92      0.82      0.87      7626

    accuracy                           0.88     16599
   macro avg       0.89      0.88      0.88     16599
weighted avg       0.89      0.88      0.88     16599

